In [ ]:
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from natsort import natsorted
import base64
import os
import shutil
import soundfile as sf
from natsort import natsorted
import textract
import textgrids
#import subprocess
import json

In [ ]:
sujs = os.listdir("C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\RAW\\")

In [ ]:
sujs[50::]

In [ ]:
suj = 'subj20'

In [ ]:
path = "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\RAW\\"+suj+".txt"

In [ ]:
# import data as bytes
text = textract.process(path)

In [ ]:
# split data by components 10 is training (should be 11 in your data as I added a component)
len(text.splitlines())

In [ ]:
trainingdata = json.loads(text.splitlines()[11].decode('utf-8'))

# Extract RAW DATA

In [ ]:
#####################################
# CONCATENATE DATA from all 4 blocks#
#####################################

block1data = json.loads(text.splitlines()[12].decode('utf-8'))
block2data = json.loads(text.splitlines()[13].decode('utf-8'))
block3data = json.loads(text.splitlines()[14].decode('utf-8'))
block4data = json.loads(text.splitlines()[15].decode('utf-8'))
allblocks = np.concatenate((block1data,block2data,block3data,block4data))

In [ ]:
##############################################
# create containers for each subtype of trial#
##############################################
RR1s =[]
GAPs = []
CONVs = []
RR2s = []

# retrive trials based on their id
trial_types = [['read&repeat1block1','read&repeat1block2','read&repeat1block3','read&repeat1block4'],
               ['gap_answerblock1','gap_answerblock2','gap_answerblock3','gap_answerblock4'],
               ['convergenceblock1','convergenceblock2','convergenceblock3','convergenceblock4'],
               ['read&repeat2block1','read&repeat2block2','read&repeat2block3','read&repeat2block4']]
trial_containers = [RR1s, GAPs,CONVs,RR2s]

for num,trial in enumerate(trial_types):
    for numi in range(len(allblocks)):
        try:
            if allblocks[numi]['trial_id'] in trial:
                trial_containers[num].append(allblocks[numi])
        except: 
            pass

In [ ]:
#############################################################
# make sure we have the desired number of trials per subtype#
#############################################################

assert(len(RR1s)==24) #6*4
assert(len(RR2s)==24) #6*4
assert(len(CONVs)==32) # 8*4

In [ ]:
# read in presented stimuli properties
stim_info = pd.read_csv("C:\\Users\\Jeremy\\Documents\\RESEARCH\\PBS_2025\\CONVERGENCE3\\STIMULI\\stimuli_AR.csv")
# create dictionnary to convert filtering properties to experimental conditions
conversion = {  '5.5_Inf': 'Normal_temporal',
                'Inf_0.35': 'Normal_spectral' ,  
                '23_Inf': 'Fast_temporal',
                'Inf_1.5': 'Fast_spectral'} 

In [ ]:
#########################################
# get names and contents for CONVERGENCE#
#########################################

names_conv = []
content_conv = []
stims_namesconv = []
conds2 = []
blocks2 = []
expcond = []
ars = []
shorts = []
for num in range(len(CONVs)):
    # get current condition
    curxcond = CONVs[num]['presented_stim'].split('/')[1].split('filter_')[1]
    # get current speed
    spee = conversion[curxcond].split('_')[0]
    # get AR from presented stim
    curar = stim_info.loc[(stim_info['sentences'] == CONVs[num]['stimulus'].split("25px\'>")[1].split("</p>")[0].lower())&(stim_info['speed'] == spee )]['articulation_rate'].values[0]
    names_conv.append('conv_'+str(num+1)+'.wav')
    content_conv.append(CONVs[num]['response'])
    stims_namesconv.append(CONVs[num]['stimulus'].split("25px\'>")[1].split("</p>")[0])
    conds2.append('conv')
    blocks2.append(CONVs[num]['trial_id'][-1])
    expcond.append(curxcond)
    ars.append(curar)
    shorts.append(stim_info.loc[(stim_info['sentences'] == CONVs[num]['stimulus'].split("25px\'>")[1].split("</p>")[0].lower())&(stim_info['speed'] == spee )]['short_names'].values[0])

dico2 = {
    'trial_type':conds2,
     'audios_names':names_conv,
    'audios_contents': content_conv,
    'stims_names': stims_namesconv,
    'blocks': blocks2,
    'conditions':expcond,
    'stim_AR': ars,
    'short_names':shorts}
df2 = pd.DataFrame.from_dict(dico2)


#################################
# get names and contents for RR1#
#################################

names_rr1 = []
content_rr1 = []
stims_names = []
conds = []
blocks = []
expcond2 = []
ars2 =[]
shorts2 = []
for num in range(len(RR1s)):
    curb = RR1s[num]['trial_id'][-1]
    names_rr1.append('rr1_'+str(num+1)+'.wav')
    content_rr1.append(RR1s[num]['response'])
    stims_names.append(RR1s[num]['stim_name'])
    conds.append('r&r1')
    blocks.append(curb)
    expcond2.append(df2.loc[df2["blocks"]==str(curb)]['conditions'].values[0])
    ars2.append(np.nan)
    shorts2.append(np.nan)

dico = {
    'trial_type':conds,
     'audios_names':names_rr1,
    'audios_contents': content_rr1,
    'stims_names': stims_names,
    'blocks':blocks,
    'conditions':expcond2,
    'stim_AR': ars2,
    'short_names':shorts2}

df1 = pd.DataFrame.from_dict(dico)

#################################
# get names and contents for RR2#
################################
names_rr2 = []
content_rr2 = []
stims_names3 = []
conds3 = []
blocks3 = []
expcond3 = []
ars3 = []
shorts3 = []
for num in range(len(RR2s)):
    curb2 = RR2s[num]['trial_id'][-1]
    names_rr2.append('rr2_'+str(num+1)+'.wav')
    content_rr2.append(RR2s[num]['response'])
    stims_names3.append(RR2s[num]['stim_name'])
    conds3.append('r&r2')
    blocks3.append(curb2)
    expcond3.append(df2.loc[df2["blocks"]==str(curb2)]['conditions'].values[0])
    ars3.append(np.nan)
    shorts3.append(np.nan)


dico3 = {
    'trial_type':conds3,
     'audios_names':names_rr2,
    'audios_contents': content_rr2,
    'stims_names': stims_names3,
    'blocks':blocks3,
    'conditions':expcond3,
    'stim_AR': ars3,
    'short_names':shorts3}

df3 = pd.DataFrame.from_dict(dico3)

#######################
# concatenate all data#
#######################
alldata = pd.concat((df1,df2,df3)).reset_index()

############################################################
# add experimental conditions based on filtering properties#
############################################################
speeds = []
filterings = []
for i in range(len(alldata)):
    curcond = alldata.iloc[i]['conditions']
    filterings.append(conversion[curcond].split('_')[1])
    speeds.append(conversion[curcond].split('_')[0])
alldata['speed'] = speeds
alldata['filtering'] = filterings


In [ ]:
# get behavioral performance for gap in speech
gap_reps = [i['sound'].split('/')[1].split('.')[0] for i in GAPs]
perc_correct  = (np.sum([1 if i == 'good' else 0 for i in gap_reps])/len(gap_reps))*100
print('Subject performance on gap trials: '+str(np.round(perc_correct,2))+'%')

In [ ]:
# save individual dataframe
finaldat = alldata[['index', 'trial_type', 'audios_names','short_names', 'stims_names','blocks', 'conditions', 'stim_AR', 'speed', 'filtering']]
outpathOne = "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\INFO\\"
if not os.path.exists(outpathOne):
    os.makedirs(outpathOne)  

finaldat.to_csv(outpathOne+suj+"_info.csv", index = None)


# Create audio files

In [ ]:
# create the folder for the audio files if it does not exist
audio_outpath = "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\audio_responses\\"
if not os.path.exists(audio_outpath):
    os.makedirs(audio_outpath)  

In [ ]:
# aggregate all the names of the audio files
final_names = alldata['audios_names'].values
# aggregate all the audio files
final_audio = alldata['audios_contents'].values

In [ ]:
# Translate each succession of weird characters into .wav file
for i,(name,audio) in enumerate(zip(final_names,final_audio)):
    decodedData = base64.b64decode(audio)
    webmfile = audio_outpath+"temp.webm"
    with open(webmfile, 'wb') as file:
           file.write(decodedData)
    newtempath = str(webmfile).replace(os.path.sep, '/')
    newfinalfilepath = str(audio_outpath+name).replace(os.path.sep, '/')
    curcommand = 'ffmpeg -i '+newtempath+' ' + newfinalfilepath
    os.system(curcommand)
    os.remove(audio_outpath+"temp.webm") 


# Create webmaus files

In [ ]:
# determine path for webmauss files
outpath = "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\webmauss\\"
inpath = "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\audio_responses\\"

In [ ]:
# create for each audio file, a text file with exactly the same name (except the end)
# write in each .txt file the transcription of each sentence

# loop across all audio files
for i in range(len(alldata)): 
    # get name of each audi file
    name = alldata.iloc[i]['audios_names'].split('.wav')[0]
    # get transcription of each sentence
    transcript = alldata.iloc[i]['stims_names']
    # get .wav file and copy it to new directory
    if not os.path.exists(outpath):
        os.makedirs(outpath)  
    shutil.copy(inpath+ name+'.wav', outpath+name+'.wav')
    # create the .txt file containing the written sentence
    with open(outpath+ name +'.txt', 'w') as f:
        f.write(transcript)

# Import .wav and .txt file within the web service web mauss

In [ ]:
# go to --> https://clarin.phonetik.uni-muenchen.de/BASWebServices/interface
# click --> Pipeline without ASR
# Pipeline name (required) --> select: G2P—>MAUS —>PHO2SYL 
# Language (required) -->  select: English (GB)
# Output format (required) --> select: Praat (.TextGrid)

# drop all audio and text files in the top rectangle (ctrl+a to select all within you audio folder)
# run the web service
# download the zip folder containing the .TextGrid files
# extract them within the folder of origin of the audio and txt files

# Retrieve duration and number of linguistic units

In [ ]:
df = pd.read_csv(outpathOne +suj+"_info.csv")
gridpath =  "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\webmauss\\"

assert(len(os.listdir(gridpath))==240)


full_durations = []
for i in range(len(df)):
    file = df.iloc[i]['audios_names']
    x, sr = sf.read(gridpath+file)
    
    #print(len(x))
    # get duration of the file
    full_durations.append(len(x)/sr)
# add lenght to the dataframe
df['full_duration'] = full_durations
# select only textgrid files
all_files = os.listdir(gridpath)  
grid_files = natsorted( list(filter(lambda f: f.endswith('.TextGrid'), all_files)))

In [ ]:
# automatically extract information from the TextGrid files
# select tiers containing number of words ,syllables and phonemes
nb_sylls = []
nb_phons = []
nb_words = []
len_sil = []
names = []
syll_dur = []
#loop across files
for i in range(len(df)):
    file = df.iloc[i]['audios_names'].split('.wav')[0] + '.TextGrid'
    if os.path.exists(gridpath+file):
        grid = textgrids.TextGrid(gridpath+file)

        # retrieve number of words
        words = []
        for i in (grid['ORT-MAU']):
            if i.text != '':
                words.append(i.text)
        nb_words.append(len(words))

        # retrieve number of syllables
        sylls = []
        syllsdur = []

        for i in (grid['MAS']):
            if i.text != '<p:>':
                sylls.append(i.text)
                syllsdur.append(i.xmax -i.xmin)
        nb_sylls.append(len(sylls))
        syll_dur.append(np.sort(syllsdur)[::-1][0:3])
        names.append(file.split(".")[0])

        # retrieve number of phonemes
        phons = []
        for i in (grid['MAU']):
            if i.text != '<p:>':
                phons.append(i.text)
        nb_phons.append(len(phons))

        # retrieve the total duration of silent part within each file
        silences = []
        for i in (grid['MAS']):
            if i.text == '<p:>':
                silences.append(i.xmax -i.xmin)
        len_sil.append(np.sum(np.asarray(silences)))
    else:
        nb_sylls.append(np.nan)
        nb_phons.append(np.nan)
        nb_words.append(np.nan)
        len_sil.append(np.nan)

# adding nb of linguist unit in the dataframe
df['nb_words'] = nb_words
df['nb_sylls'] = nb_sylls
df['nb_phons'] = nb_phons
df['silent_duration'] = len_sil
df['phonation_time'] = df['full_duration'].values - df['silent_duration'].values
df['articulation rate'] = df['nb_sylls'].values/df['phonation_time'].values
#df.to_csv(outpath+'info_final2.csv', index =None)

#df2.to_csv(outpath+'webmausserrors.csv', index =None) 

In [ ]:
df3 = df.loc[df['trial_type']=='conv'].reset_index()

In [ ]:
arcs = []
for i in range(len(df3)):
    c = df3.iloc[i]['conditions']
    arc = df.loc[(df['trial_type']=="r&r1")&(df['conditions']==c)]['articulation rate'].mean()
    arcs.append(arc)
df3['baseline_AR'] = arcs
df3['absdiff'] = abs(df3['baseline_AR'] - df3['articulation rate'])
df3['subj'] = np.repeat(suj,len(df3))
df3.to_csv("CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\FOR_R\\"+suj+'_convergence.csv',index=None)

# Once all participants data have been processed 

In [ ]:
# create mega dataframe for r analysis
rpath = "\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\FOR_R\\"
files = os.listdir(rpath)
len(files)

In [ ]:
bigos = []

for file in files:
    dt = pd.read_csv(rpath+file)
    bigos.append(dt)
bg = pd.concat(bigos)
bg.to_csv("df_for_r_analysis.csv",index=None)